# A320 Optimization - AirFrance/KLM Project

Group 5 - Aaron, Faruk, Matthieu, Julien, Nicolas

In [43]:
# Necessary packages
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import *
from processing_py import *
import csv
import pandas as pd
import time

In [44]:
nb_places=203
nb_row=29
nb_column=7

In [45]:
## Fonctions de passge d'une représentation à une autre

def switch_lin_to_real(lin,nb_row,nb_column):
    column=lin%nb_column
    if column==0:
        column=nb_column
        row=lin//nb_column
    else:
        row=lin//nb_column+1

    real=(column,row) #colonne, rangée
    return real



def switch_real_to_lin(real,nb_row,nb_column):
    column,row=real[0],real[1]

    lin=(row-1)*nb_column+column
    return lin

## Fonction distance entre 2 passagers

def distance_grpe(real1,real2):

    ''' Calcule la distance entre un passager assis en real1 et un passager assis en real 2 avec real1[0]<=real2[0] et real1[1]<=real2[1] pour un groupe de au moins 3 personnes'''

    return real2[0]-real1[0]+2*(real2[1]-real1[1])



def distance_x(bool1,real1,real2):
    return bool1*real1[0]-real2[0]

def distance_y(bool1,real1,real2):
    return bool1*real1[1]-real2[1]

def distance_x_m(bool1,real1,real2):
    return -distance_x(bool1,real1,real2)

def distance_y_m(bool1,real1,real2):
    return -distance_y(bool1,real1,real2)



## Data Importation

In [46]:
## Importation des données initiales

ini_start=time.time()
# file name : str
info_file = "data.csv"

# InfoGrpe : dict[int : float]
InfoGrpe = {}
Passagers={}
k=1
# Chargement des donnees
with open(info_file) as InfoFile:
    reader = csv.DictReader(InfoFile)
    for row in reader:

        #Groupe_j
        j = int(row["Numero du groupe"])
        InfoGrpe[j] = {}
        InfoGrpe[j]["Composition"]=[int(row['Femmes']),int(row['Hommes']),int(row['Enfants'])]
        InfoGrpe[j]["Nbre total"]=int(row['Tot'])
        InfoGrpe[j]["Classe"]=row['Classe']

        if InfoGrpe[j]["Classe"]=='J':
            nb_classe_affaire+=InfoGrpe[j]["Nbre total"]
        t_str=row["TransitTime"]
        if t_str=="00:00:00":
            t=10**6
        else:
            time_list=[int(x) for x in t_str.split(":")]
            t=time_list[0]*60+time_list[1]
        InfoGrpe[j]["TransitT"]=time
        InfoGrpe[j]["Passagers"]=[]

        #Passagers_groupe_j

        #Enfants
        k_p=k+int(row['Enfants'])
        for x in range(k,k_p):
            Passagers[x]={}
            Passagers[x]["Groupe"]=j
            Passagers[x]["Poids"]=35
            InfoGrpe[j]["Passagers"].append(x)
        k=k_p

        #Hommes
        k_p+=int(row['Hommes'])
        for x in range(k,k_p):
            Passagers[x]={}
            Passagers[x]["Groupe"]=j
            Passagers[x]["Poids"]=85
            InfoGrpe[j]["Passagers"].append(x)

        k=k_p

        #Femmes
        k_p+=int(row['Femmes'])
        for x in range(k,k_p):
            Passagers[x]={}
            Passagers[x]["Groupe"]=j
            Passagers[x]["Poids"]=70
            InfoGrpe[j]["Passagers"].append(x)
        k=k_p

InfoGrpedf=pd.DataFrame(InfoGrpe)
Passagersdf=pd.DataFrame(Passagers)
nb_passagers=len(Passagers)

if nb_passagers>nb_places:
    print("Embarquement impossible, capacité de l'avion insuffisante'")

#print(InfoGrpedf)
#print(Passagersdf)




##Définition de la zone de classe affaire

try:place_max_affaire=7*math.ceil(nb_classe_affaire/4)
except: place_max_affaire=0
affaire=[]
for k in InfoGrpe.keys():
    grpe=InfoGrpe[k]
    if InfoGrpe[j]["Classe"]=='J':
        for pa in grpe["Passagers"]:
            affaire.append(pa)

normaux=[]
for k in InfoGrpe.keys():
    grpe=InfoGrpe[k]
    if InfoGrpe[j]["Classe"]=='Y':
        for pa in grpe["Passagers"]:
            normaux.append(pa)

## Définition tableau voisins et places issues de secours

V_pr={}
V_snd={}
V_ter={}
for pl in range(1,nb_places+1):
    V_pr[pl]={i: 0 for i in range(1,nb_places+1)}
    V_snd[pl]={i: 0 for i in range(1,nb_places+1)}
    V_ter[pl]={i: 0 for i in range(1,nb_places+1)}
    real=switch_lin_to_real(pl,nb_row,nb_column)
    column,row=real[0],real[1]
    vois_pr=[]
    vois_snd=[]
    vois_ter=[]


    if column in [1,5]:
        vois_pr.append(pl+1)
    if column in [2,6]:
        vois_pr.append(pl-1)
        vois_pr.append(pl+1)
    if column in [3,7]:
        vois_pr.append(pl-1)

    if row==1:
        vois_snd.append(pl+7)
    if row==29:
        vois_snd.append(pl-7)
    else:
        vois_snd.append(pl+7)
        vois_snd.append(pl-7)


    if column==3:
        vois_ter.append(pl+2)
    if column==5:
        vois_ter.append(pl-2)




    for v in vois_pr:
        V_pr[pl][v]=1
    for v in vois_snd:
        V_snd[pl][v]=1
    for v in vois_ter:
        V_ter[pl][v]=1

# Voisinnage classe affaire

V_J={}
for pl in range(1,place_max_affaire):
    V_J[pl]={i: 0 for i in range(1,place_max_affaire+1)}
    real=switch_lin_to_real(pl,nb_row,nb_column)
    column,row=real[0],real[1]
    vois_J=[]

    if column==1 or column==5:
        vois_J.append(pl+2)
    if column==3 or column==7:
        vois_J.append(pl-2)

    for v in vois_J:
        V_J[pl][v]=1

# Issues de secours

Issues=[]

for row in [11,12]:
    for column in [1,7]:
        Issues.append(switch_real_to_lin((column,row),nb_row,nb_column))


## Model Definition

In [47]:
## Définition de modèle
# m : Model
m = Model("PL")



## Définition des variables


#format linéaire
print("Définition des variables \n")
E={}

for pa in range(1,nb_passagers+1): # passagers
    for pl in range(1,nb_places+1): # places
        E[(pa,pl)]=m.addVar(vtype=GRB.BINARY)


# Définiton des variables de distances et contraintes associées
print('Définiton des variables de distances et contraintes associées \n')
R_max={}
C_max={}
R_constr={}
C_constr={}


for k in InfoGrpe.keys():
    start=time.time()
    grpe=InfoGrpe[k]
    print("Grouepe{} ={}".format(k,grpe["Passagers"]))
    if grpe["Classe"]=='J':
        R_max[k]=m.addVar(vtype=GRB.INTEGER)
        C_max[k]=m.addVar(vtype=GRB.INTEGER)
        for pa1 in grpe["Passagers"]:
            R_constr[pa1]=m.addConstr(R_max[k]>=quicksum(E[(pa1,pl1)]*switch_lin_to_real(pl1,nb_row,nb_column)[1] for pl1 in range(1,place_max_affaire+1)))
            C_constr[pa1]=m.addConstr(C_max[k]>=quicksum(E[(pa1,pl1)]*switch_lin_to_real(pl1,nb_row,nb_column)[0] for pl1 in range(1,place_max_affaire+1)))

    else:
        R_max[k]=m.addVar(vtype=GRB.INTEGER)
        C_max[k]=m.addVar(vtype=GRB.INTEGER)
        for pa1 in grpe["Passagers"]:
            R_constr[pa1]=m.addConstr(R_max[k]>=quicksum(E[(pa1,pl1)]*switch_lin_to_real(pl1,nb_row,nb_column)[1] for pl1 in range(place_max_affaire+1,nb_places+1)))
            C_constr[pa1]=m.addConstr(C_max[k]>=quicksum(E[(pa1,pl1)]*switch_lin_to_real(pl1,nb_row,nb_column)[0] for pl1 in range(place_max_affaire+1,nb_places+1)))
    laps=round(time.time()-start,3)
    print("Groupe {}={}: time spent= {}".format(k,grpe["Passagers"],laps))

## Définition de contrainte couloir et classe affaire

print("\nAdd contrainte:Couloir vide et places classe affaire vides \n")
# couloir vide
l=[]
if place_max_affaire:
    row_max=switch_lin_to_real(place_max_affaire,nb_row,nb_column)[1]
    for row in range(1,row_max+1):
        l.append(switch_real_to_lin((2,row),nb_row,nb_column))
        l.append(switch_real_to_lin((6,row),nb_row,nb_column))
for row in range(1,nb_row+1):
    l.append(switch_real_to_lin((4,row),nb_row,nb_column))
Cc=m.addConstr(quicksum(E[(pa,lin)] for pa in range(1,nb_passagers+1) for lin in l)==0)


Définition des variables 

Définiton des variables de distances et contraintes associées 

Grouepe1 =[1]
Groupe 1=[1]: time spent= 0.019
Grouepe2 =[2]
Groupe 2=[2]: time spent= 0.008
Grouepe3 =[3]
Groupe 3=[3]: time spent= 0.009
Grouepe4 =[4]
Groupe 4=[4]: time spent= 0.009
Grouepe5 =[5, 6]
Groupe 5=[5, 6]: time spent= 0.012
Grouepe6 =[7]
Groupe 6=[7]: time spent= 0.008
Grouepe7 =[8]
Groupe 7=[8]: time spent= 0.01
Grouepe8 =[9, 10]
Groupe 8=[9, 10]: time spent= 0.012
Grouepe9 =[11]
Groupe 9=[11]: time spent= 0.007
Grouepe10 =[12]
Groupe 10=[12]: time spent= 0.007
Grouepe11 =[13, 14]
Groupe 11=[13, 14]: time spent= 0.014
Grouepe12 =[15]
Groupe 12=[15]: time spent= 0.012
Grouepe13 =[16]
Groupe 13=[16]: time spent= 0.007
Grouepe14 =[17, 18]
Groupe 14=[17, 18]: time spent= 0.016
Grouepe15 =[19]
Groupe 15=[19]: time spent= 0.012
Grouepe16 =[20]
Groupe 16=[20]: time spent= 0.013
Grouepe17 =[21, 22]
Groupe 17=[21, 22]: time spent= 0.011
Grouepe18 =[23]
Groupe 18=[23]: time spent= 0.009
Grouep

### Set constraints

In [48]:
## Définition des contraintes de bases

# chaque passager est assis à une place
print("Add contrainte:chaque passager est assis à une place\n")
C1={}

for pa in range(1,nb_passagers+1): # passagers
    C1[pa]=m.addConstr(quicksum(E[(pa,pl)] for pl in range(1,nb_places+1))==1)

# chaque siège est occupé par au maximum 1 personne
print("Add contrainte:chaque siège est occupé par au maximum 1 personne\n")

C2={}

for pl in range(1,nb_places+1): # places
    C2[pl]=m.addConstr(quicksum(E[(pa,pl)] for pa in range(1,nb_passagers+1))<=1)


##  Définition contrainte transit time
'''
print("Add contrainte:transit time\n")
place_lim = nb_places//3

C3={}
for pa in range(1,nb_passagers+1): # passagers
   if InfoGrpe[Passagers[pa]["Groupe"]]["TransitT"]<=5400:    # si transit_time inférieur à 1h30
       C3[pa] = m.addConstr(quicksum([E[(pa,pl)] for pl in range(place_lim+1,nb_places+1)])==0)
'''



## Définition contrainte centrage

print("Add contrainte:barycentre\n")

#Définition des limites acceptables
column_range=[13,17]
row_range=[3,5]

#Somme des poids de l'ensemble des passages du vol
somme_poids = sum(Passagers[i]["Poids"] for i in range(1,nb_passagers+1))

#Calcul du barycentre
barycentre_column=0
barycentre_row=0
for pa in range(1,nb_passagers+1): # passagers
    for pl in range(1,nb_places+1): # places
            barycentre_column+=switch_lin_to_real(pl,nb_row,nb_column)[0]*E[(pa,pl)]*Passagers[pa]["Poids"]
            barycentre_row+=switch_lin_to_real(pl,nb_row,nb_column)[1]*E[(pa,pl)]*Passagers[pa]["Poids"]
barycentre_column/=somme_poids
barycentre_row/=somme_poids

by=m.addVar(vtype=GRB.CONTINUOUS)
bx=m.addVar(vtype=GRB.CONTINUOUS)
m.addConstr(barycentre_column==by)
m.addConstr(barycentre_row==bx)

C4={}
C4["sup column"]=m.addConstr(barycentre_column<=5)
C4["inf column"]=m.addConstr(barycentre_column>=3)
C4["sup row"]=m.addConstr(barycentre_row<=17)
C4["inf row"]=m.addConstr(barycentre_row>=13)

## Définition contrainte enfants

print("Add contrainte:Enfants\n")

C_child={}
for k in InfoGrpe.keys():
    grpe=InfoGrpe[k]
    gr=[]
    for pa1 in grpe["Passagers"]:
        if Passagers[pa1]["Poids"]!=35:
            gr.append(pa1)
    for pa1 in grpe["Passagers"]:
        if Passagers[pa1]["Poids"]==35:
            C_child[pa]=m.addConstr(quicksum(E[(pa1,pl)] for pl in Issues)==0)
            for pl in range(1,nb_places+1):
                real=switch_lin_to_real(pl,nb_row,nb_column)
                if real[0]==1 or real[0]==5:
                    C_child[pa1,pa1]=m.addConstr(quicksum(E[(pa,pl+1)] for pa in gr)>=E[(pa1,pl)])
                elif real[0]==7 or real[0]==3:
                    C_child[pa1,pa1]=m.addConstr(quicksum(E[(pa,pl-1)] for pa in gr)>=E[(pa1,pl)])
                else:
                    C_child[pa1,pa1]=m.addConstr(quicksum(E[(pa,pl-1)]+E[(pa,pl+1)] for pa in gr)>=E[(pa1,pl)])





## Définition contrainte classe


if place_max_affaire:
    print("Add contrainte: Position classe affaire\n")

    C_J={}
    for k in InfoGrpe.keys():
        grpe=InfoGrpe[k]
        if grpe['Classe']=="J":
            for pa1 in grpe["Passagers"]:
                m.addConstr(quicksum(E[(pa,pl)] for pl in range(1,place_max_affaire+1))==1)
## Définition contraintes WHCR

print("Add contrainte: WHCR\n")
D_H={}
D_B={}
C_WHCR={}
for pa in range(1,nb_passagers+1):
    if Passagers[pa]["Poids"]==100:
        C_WHCR[pa,1]=m.addConstr(quicksum(E[pa,switch_real_to_lin((3,row),nb_row,nb_column)]+E[pa,switch_real_to_lin((5,row),nb_row,nb_column)] for row in range(1,nb_row+1) )==1)
        for c in [3,5]:
            for row in range(1,nb_row+1):
                switch_real_to_lin((c,row),nb_row,nb_column)
                C_WHCR[pa,2]=m.addConstr(quicksum(E[(pa1,pl+1)]+E[(pa1,pl-1)] for pa1 in range(1,nb_passagers+1))<=1-E[(pa,pl)])
                if row==nb_row:
                    C_WHCR[pa,3]=m.addConstr(quicksum(E[(pa1,pl-7)] for pa1 in range(1,nb_passagers+1))<=1-E[(pa,pl)])
                    C_WHCR[pa,4]=m.addConstr(quicksum(E[(pa1,pl-8)]+E[(pa1,pl-6)] for pa1 in range(1,nb_passagers+1))<=E[(pa,pl-7)])

                elif row==1:
                    C_WHCR[pa,3]=m.addConstr(quicksum(E[(pa1,pl+7)] for pa1 in range(1,nb_passagers+1))<=1-E[(pa,pl)])
                    C_WHCR[pa,4]=m.addConstr(quicksum(E[(pa1,pl+8)]+E[(pa1,pl+6)] for pa1 in range(1,nb_passagers+1))<=E[(pa,pl+7)])

                else:
                    D_H[c,row]=m.addVar(vtype=GRB.BINARY)
                    D_B[c,row]=m.addVar(vtype=GRB.BINARY)
                    C_WHCR[c,row,1]=m.addConstr(D_H[c,row]>=1/3*(E[(pa,pl+6)]+E[(pa,pl+7)]+E[(pa,pl+8)]))
                    C_WHCR[c,row,2]=m.addConstr(D_H[c,row]<=(E[(pa,pl+6)]+E[(pa,pl+7)]+E[(pa,pl+8)]))
                    C_WHCR[c,row,3]=m.addConstr(D_B[c,row]>=1/3*(E[(pa,pl-6)]+E[(pa,pl-7)]+E[(pa,pl-8)]))
                    C_WHCR[c,row,4]=m.addConstr(D_B[c,row]<=(E[(pa,pl-6)]+E[(pa,pl-7)]+E[(pa,pl-8)]))
                    C_WHCR[pa,3]=m.addConstr(D_H[c,row]+D_B[c,row]<=2-E[(pa,pl)])

Add contrainte:chaque passager est assis à une place

Add contrainte:chaque siège est occupé par au maximum 1 personne

Add contrainte:barycentre

Add contrainte:Enfants

Add contrainte: WHCR



### Set objective function

In [49]:
## Définition de la fonction objectif


for w1 in [0.4+1/30]:#np.linspace(0.4,0.5,10)
    w2=1-w1
    Dist=0

    for k in InfoGrpe.keys():
        grpe=InfoGrpe[k]
        if grpe["Classe"]=='J':
            for pa1 in grpe["Passagers"]:
                Dist+=w2*(R_max[k]-quicksum(E[(pa1,pl1)]*switch_lin_to_real(pl1,nb_row,nb_column)[1] for pl1 in range(1,place_max_affaire+1)))+w1*(C_max[k]-quicksum(E[(pa1,pl1)]*switch_lin_to_real(pl1,nb_row,nb_column)[0] for pl1 in range(1,place_max_affaire+1)))
        else:
            for pa1 in grpe["Passagers"]:
                Dist+=w2*(R_max[k]-quicksum(E[(pa1,pl1)]*switch_lin_to_real(pl1,nb_row,nb_column)[1] for pl1 in range(place_max_affaire+1,nb_places+1)))+w1*(C_max[k]-quicksum(E[(pa1,pl1)]*switch_lin_to_real(pl1,nb_row,nb_column)[0] for pl1 in range(place_max_affaire+1,nb_places+1)))


    # -- Choix d'un paramétrage d'optimisation --
    m.params.outputflag=1
    #m.params.MIPGapAbs=40
    m.params.TimeLimit=600
    m.setObjective(Dist,GRB.MINIMIZE)

    lap_ini=time.time()-ini_start
    print("Temps d'initialisation: {}\n".format(lap_ini))


Parameter outputflag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Temps d'initialisation: 6.824764013290405



In [27]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 868 rows, 31078 columns and 252347 nonzeros
Model fingerprint: 0x8d23dcdd
Variable types: 0 continuous, 31078 integer (30856 binary)
Coefficient statistics:
  Matrix range     [3e-03, 3e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 304.3000000
Presolve removed 217 rows and 4564 columns
Presolve time: 0.57s
Presolved: 651 rows, 26514 columns, 150550 nonzeros
Variable types: 0 continuous, 26514 integer (26444 binary)

Root relaxation: objective -4.618528e-14, 7009 iterations, 1.60 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.00000    0  173  304.30000   -0.00000   100%     -    4s
H    0     0                   

In [28]:
print(m.Status)

9


In [29]:
## Notation du résultat

Sat=0
for k in InfoGrpe.keys():
    Dist=0
    S=0
    grpe=InfoGrpe[k]
    if grpe["Classe"]=='J':
        for pa1 in grpe["Passagers"][1:]:
            for pl1 in range(1,place_max_affaire+1): # places
                Dist+=w2*(R_max[k].x-quicksum(E[(pa1,pl1)].x*switch_lin_to_real(pl1,nb_row,nb_column)[1] for pl1 in range(1,place_max_affaire+1)))+w1*(C_max[k].x-quicksum(E[(pa1,pl1)].x*switch_lin_to_real(pl1,nb_row,nb_column)[0] for pl1 in range(1,place_max_affaire+1)))

        for pa1 in grpe["Passagers"]:
            for pa2 in grpe["Passagers"]:
                for pl1 in range(1,place_max_affaire+1): # places
                    for pl2 in range(max(pl1-7,1),min(pl1+8,place_max_affaire+1)):
                        if grpe["Nbre total"]>3:
                            if V_snd[pl1][pl2]:
                                Sat+=0.5*E[(pa1,pl1)].x*E[(pa2,pl2)].x
                                S+=0.5*E[(pa1,pl1)].x*E[(pa2,pl2)].x
                            if V_ter[pl1][pl2]:
                                Sat+=0.5*E[(pa1,pl1)].x*E[(pa2,pl2)].x
                                S+=0.5*E[(pa1,pl1)].x*E[(pa2,pl2)].x
                        if grpe["Nbre total"]<=3:
                            if V_pr[pl1][pl2]:
                                Sat+=E[(pa1,pl1)].x*E[(pa2,pl2)].x
                                S+=E[(pa1,pl1)].x*E[(pa2,pl2)].x

    else:
        for pa1 in grpe["Passagers"][1:]:
            for pl1 in range(place_max_affaire+1,nb_places+1): # places
                Dist+=w2*(R_max[k].x-quicksum(E[(pa1,pl1)].x*switch_lin_to_real(pl1,nb_row,nb_column)[1] for pl1 in range(place_max_affaire+1,nb_places+1)))+w1*(C_max[k].x-quicksum(E[(pa1,pl1)].x*switch_lin_to_real(pl1,nb_row,nb_column)[0] for pl1 in range(place_max_affaire+1,nb_places+1)))

        for pa1 in grpe["Passagers"]:
            for pa2 in grpe["Passagers"]:
                for pl1 in range(place_max_affaire+1,nb_places+1): # places
                    for pl2 in range(max(pl1-7,place_max_affaire+1),min(pl1+8,nb_places+1)):
                        if grpe["Nbre total"]>3:
                            if V_snd[pl1][pl2]:
                                Sat+=0.5*E[(pa1,pl1)].x*E[(pa2,pl2)].x
                                S+=0.5*E[(pa1,pl1)].x*E[(pa2,pl2)].x
                            if V_ter[pl1][pl2]:
                                Sat+=0.5*E[(pa1,pl1)].x*E[(pa2,pl2)].x
                                S+=0.5*E[(pa1,pl1)].x*E[(pa2,pl2)].x

                        if V_pr[pl1][pl2]:
                            Sat+=E[(pa1,pl1)].x*E[(pa2,pl2)].x
                            S+=E[(pa1,pl1)].x*E[(pa2,pl2)].x
    print("Grpe {} Distance= {} / Score= {}".format(k,Dist,S))

print("Score total={}".format(Sat))
print("Distance totale={}".format(m.ObjVal))

Grpe 1 Distance= 0 / Score= 0.0
Grpe 2 Distance= 0 / Score= 0.0
Grpe 3 Distance= 0 / Score= 0.0
Grpe 4 Distance= 0 / Score= 0.0
Grpe 5 Distance= <gurobi.LinExpr: 0.0> / Score= 2.0
Grpe 6 Distance= 0 / Score= 0.0
Grpe 7 Distance= 0 / Score= 0.0
Grpe 8 Distance= <gurobi.LinExpr: 0.0> / Score= 2.0
Grpe 9 Distance= 0 / Score= 0.0
Grpe 10 Distance= 0 / Score= 0.0
Grpe 11 Distance= <gurobi.LinExpr: 87.96666666666664> / Score= 2.0
Grpe 12 Distance= 0 / Score= 0.0
Grpe 13 Distance= 0 / Score= 0.0
Grpe 14 Distance= <gurobi.LinExpr: 0.0> / Score= 2.0
Grpe 15 Distance= 0 / Score= 0.0
Grpe 16 Distance= 0 / Score= 0.0
Grpe 17 Distance= <gurobi.LinExpr: 0.0> / Score= 2.0
Grpe 18 Distance= 0 / Score= 0.0
Grpe 19 Distance= <gurobi.LinExpr: 263.90000000000083> / Score= 5.0
Grpe 20 Distance= <gurobi.LinExpr: 202.9999999999992> / Score= 5.0
Grpe 21 Distance= <gurobi.LinExpr: 115.03333333333316> / Score= 2.0
Grpe 22 Distance= 0 / Score= 0.0
Grpe 23 Distance= <gurobi.LinExpr: 87.96666666666664> / Score= 2.

## Display Results

In [41]:
class A320:
    
    def __init__(self,nb_rows,nb_columns,nb_places,nb_passengers):

        self.app = App(1709,500)    
        A320_seats = self.app.loadImage('C:\\Users\\Faruk\\Desktop\\code\\A320_optimization\\A320.jpg')
        self.app.image(A320_seats,0,0)
        self.app.redraw()
        self.color = np.random.choice(range(256), size=(174,3))

        self.nb_rows = nb_rows
        self.nb_columns = nb_columns
        self.nb_places = nb_places
        self.nb_passengers = nb_passengers

    def draw_passenger(self,row, column, group, category):

        # Fill with group color
        self.app.fill(self.color[group][0],self.color[group][1],self.color[group][2])
        # Stroke with caterory color
        
        self.app.stroke(0,0,0)
        # Find x,y in the drawing
        if column <= 10:
            x = 322+column*30
        elif column == 11:
            x = 333+column*30
        elif column <= 16:
            x = 342+column*30
        else:
            x = 344+column*30
        if row<=3:
            y = 126+row*20
        else:
            y = 136+row*20

        if category == 'Child':
            self.app.ellipse(x,y,10,10)
        else:
            self.app.ellipse(x,y,15,15)

        self.app.redraw()

    def draw_center_of_gravity(self,bx,by):

        self.app.fill(255,0,0)
        self.app.noStroke()
        self.app.ellipse(342+bx*30,141+by*20,5,5)
        self.app.text('CG',350+bx*30,141+by*20)
        self.app.redraw()

    def show_result(self, E, passengers, bx, by):

        for pa in range(1,self.nb_passengers+1): # passagers
            for pl in range(1,self.nb_places+1): # places
                if E[(pa,pl)].x:
                    row, column = switch_lin_to_real(pl,self.nb_rows,self.nb_columns)

                    category = ''
                    if passengers[pa]["Poids"] == '35':
                        category = 'Child'
                
                    self.draw_passenger(row,column,passengers[pa]["Groupe"],category)
        
        self.draw_center_of_gravity(bx,by)
        
        self.app.redraw()

In [42]:
A320 = A320(nb_row,nb_column,nb_places,nb_passagers)
A320.show_result(E,Passagers,bx.x,by.x)

Starting App...
>> [Jython] Created!



In [82]:
#A320.app.exit()